In [1]:
import re
from typing import NamedTuple, Dict, List, Tuple
from collections import defaultdict

import numpy as np
import networkx as nx
import igraph as ig

import leidenalg
import math

from sklearn.metrics import normalized_mutual_info_score
from tqdm.notebook import tqdm

from utils import nxBuildGraph, from_ig_to_nx_partition_multiplex, from_part_to_list, fusion_graph

In [2]:
DATA_STORAGE = r'../Data/'
FILE_NAMES = ['candida_genetic', 'drosophila_genetic', 'homo_genetic', 'mus_genetic']

### 1. Data Reading and Processing

In [3]:
dictionary = {}
layers_info = {}
for _, file in enumerate(FILE_NAMES):
    with open(DATA_STORAGE + file + '_layers.txt') as f:
        lines = f.readlines()[1:]
    layers = {}
    if _ == 0:
        for idx, layer in enumerate(lines):
            dictionary[re.sub(r'[\s\d]', '', layer)] = idx

    for idx, layer in enumerate(lines):
        layers[idx] = dictionary[re.sub(r'[\s\d]', '', layer)]
    layers_info[file] = layers

In [4]:
graphs_multiplex_edges_list = {}
for _, file in enumerate(FILE_NAMES):
    with open(DATA_STORAGE + file + '_multiplex.edges') as f:
        lines = f.readlines()
    edges = {}
    for row in lines:
        edge = re.findall(r'\d+', row)
        
        key = layers_info[file][int(edge[0])-1]
        edge = [int(i) for i in edge[1:-1]]
        if key not in edges.keys():
            edges[key] = []
        edges[key].append(edge)
        
    graphs_multiplex_edges_list[file] = edges

In [5]:
graphs_multiplex_nx = {}
for name in graphs_multiplex_edges_list:
    graphs_multiplex_nx[name] = nxBuildGraph(graphs_multiplex_edges_list[name])

In [6]:
graphs_multiplex_ig = {}
for name in graphs_multiplex_nx:
    graphs_multiplex_ig[name] = list(map(lambda x: ig.Graph.from_networkx(x), graphs_multiplex_nx[name])) 

### 2. Define Brute Force Loop

In [7]:
from itertools import combinations, combinations_with_replacement, permutations
step_size = 10
alph_count = 7

alphas = np.asarray(list(combinations_with_replacement([i for i in range(step_size)], alph_count)))
alphas = alphas[alphas.sum(1) == step_size]

permutation_len = math.factorial(alph_count)
full_alphas = np.zeros((alphas.shape[0] * permutation_len, alph_count))
for _, i in enumerate(alphas):
    full_alphas[_*permutation_len : (_+1)*permutation_len, :] = np.asarray(list(permutations(i, alph_count)))
    
full_alphas = full_alphas / step_size
full_alphas = np.unique(full_alphas, axis = 0)

### 3. Brute Force

### 3. 1. candida_genetic

In [8]:
gt_partition, _ =  leidenalg.find_partition_multiplex(
graphs_multiplex_ig['candida_genetic'],  # list из слоев графа в формате igraph
leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
n_iterations=2, 
max_comm_size=0, 
seed=None)




nmi_arr = []
net = graphs_multiplex_nx['candida_genetic']
for alpha in tqdm(full_alphas):
    adj_matrixes = [nx.adjacency_matrix(i, i.nodes()) for i in net]
    alpha_adj_matrixes = [i*j for i,j in zip(adj_matrixes, alpha)]
    
    adj_for_graph = alpha_adj_matrixes[0]
    for i in range(1, len(alpha_adj_matrixes)):
        adj_for_graph += alpha_adj_matrixes[i]
        
    alpha_graph = nx.from_scipy_sparse_matrix(adj_for_graph, parallel_edges=False, edge_attribute='weight')
    alpha_graph_ig = ig.Graph.from_networkx(alpha_graph)
    
    part = leidenalg.find_partition(alpha_graph_ig, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(gt_partition, part)
    nmi_arr.append(nmi)

  0%|          | 0/8001 [00:00<?, ?it/s]

In [13]:
np.max(nmi_arr)

0.9245790811463562

In [15]:
max_alphas = np.unique(full_alphas[np.nonzero(np.array(nmi_arr) == np.max(nmi_arr))], axis=0)[:100, :]
fusion_k = max_alphas.mean(0)

In [18]:
max_alphas[0]

array([0.1, 0.1, 0.1, 0. , 0.1, 0.1, 0.5])

In [17]:
for net_name in graphs_multiplex_nx:
    net = graphs_multiplex_nx[net_name]
    
    part_gt , _ =  leidenalg.find_partition_multiplex(
    graphs_multiplex_ig[net_name],  # list из слоев графа в формате igraph
    leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
    n_iterations=2, 
    max_comm_size=0, 
    seed=None)
    
    
    ig_graph = fusion_graph(net, max_alphas[0])
    part = leidenalg.find_partition(ig_graph, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(part_gt, part)
    print(nmi)

0.9278145450421957
0.30425818941391575
0.2812681362121794
0.6234527544161886


### 3.2. drosophila_genetic

In [28]:
gt_partition, _ =  leidenalg.find_partition_multiplex(
graphs_multiplex_ig['drosophila_genetic'],  # list из слоев графа в формате igraph
leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
n_iterations=2, 
max_comm_size=0, 
seed=None)




nmi_arr = []
net = graphs_multiplex_nx['drosophila_genetic']
for alpha in tqdm(full_alphas):
    adj_matrixes = [nx.adjacency_matrix(i, i.nodes()) for i in net]
    alpha_adj_matrixes = [i*j for i,j in zip(adj_matrixes, alpha)]
    
    adj_for_graph = alpha_adj_matrixes[0]
    for i in range(1, len(alpha_adj_matrixes)):
        adj_for_graph += alpha_adj_matrixes[i]
        
    alpha_graph = nx.from_scipy_sparse_matrix(adj_for_graph, parallel_edges=False, edge_attribute='weight')
    alpha_graph_ig = ig.Graph.from_networkx(alpha_graph)
    
    part = leidenalg.find_partition(alpha_graph_ig, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(gt_partition, part)
    nmi_arr.append(nmi)

  0%|          | 0/8001 [00:00<?, ?it/s]

In [31]:
max_alphas = np.unique(full_alphas[np.nonzero(np.array(nmi_arr) == np.max(nmi_arr))], axis=0)[:100, :]
fusion_k = max_alphas.mean(0)

In [ ]:
# [0.4, 0. , 0.1, 0.1, 0. , 0.1, 0.3]

In [32]:
fusion_k

array([0.4, 0. , 0.1, 0.1, 0. , 0.1, 0.3])

In [33]:
for net_name in graphs_multiplex_nx:
    net = graphs_multiplex_nx[net_name]
    
    part_gt , _ =  leidenalg.find_partition_multiplex(
    graphs_multiplex_ig[net_name],  # list из слоев графа в формате igraph
    leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
    n_iterations=2, 
    max_comm_size=0, 
    seed=None)
    
    
    ig_graph = fusion_graph(net, fusion_k)
    part = leidenalg.find_partition(ig_graph, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(part_gt, part)
    print(nmi)

0.8068880094276362
0.5371332694052983
0.4720083219388514
0.6082265736876056


### 3.3. homo_genetic

In [48]:
gt_partition, _ =  leidenalg.find_partition_multiplex(
graphs_multiplex_ig['homo_genetic'],  # list из слоев графа в формате igraph
leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
n_iterations=2, 
max_comm_size=0, 
seed=None)




nmi_arr = []
net = graphs_multiplex_nx['homo_genetic']
for alpha in tqdm(full_alphas):
    adj_matrixes = [nx.adjacency_matrix(i, i.nodes()) for i in net]
    alpha_adj_matrixes = [i*j for i,j in zip(adj_matrixes, alpha)]
    
    adj_for_graph = alpha_adj_matrixes[0]
    for i in range(1, len(alpha_adj_matrixes)):
        adj_for_graph += alpha_adj_matrixes[i]
        
    alpha_graph = nx.from_scipy_sparse_matrix(adj_for_graph, parallel_edges=False, edge_attribute='weight')
    alpha_graph_ig = ig.Graph.from_networkx(alpha_graph)
    
    part = leidenalg.find_partition(alpha_graph_ig, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(gt_partition, part)
    nmi_arr.append(nmi)

  0%|          | 0/8001 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [50]:
max_alphas = np.unique(full_alphas[np.nonzero(np.array(nmi_arr) == np.max(nmi_arr))], axis=0)[:100, :]
fusion_k = max_alphas.mean(0)

In [51]:
max_alphas

array([[0. , 0. , 0.7, 0.2, 0. , 0.1, 0. ]])

In [52]:
for net_name in graphs_multiplex_nx:
    net = graphs_multiplex_nx[net_name]
    
    part_gt , _ =  leidenalg.find_partition_multiplex(
    graphs_multiplex_ig[net_name],  # list из слоев графа в формате igraph
    leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
    n_iterations=2, 
    max_comm_size=0, 
    seed=None)
    
    
    ig_graph = fusion_graph(net, max_alphas[0])
    part = leidenalg.find_partition(ig_graph, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(part_gt, part)
    print(nmi)

0.4458520288581317
0.5430034943830734
0.4890599368349828
0.6088496749480669


### 3.4. mus_genetic

In [36]:
gt_partition, _ =  leidenalg.find_partition_multiplex(
graphs_multiplex_ig['mus_genetic'],  # list из слоев графа в формате igraph
leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
n_iterations=2, 
max_comm_size=0, 
seed=None)




nmi_arr = []
net = graphs_multiplex_nx['mus_genetic']
for alpha in tqdm(full_alphas):
    adj_matrixes = [nx.adjacency_matrix(i, i.nodes()) for i in net]
    alpha_adj_matrixes = [i*j for i,j in zip(adj_matrixes, alpha)]
    
    adj_for_graph = alpha_adj_matrixes[0]
    for i in range(1, len(alpha_adj_matrixes)):
        adj_for_graph += alpha_adj_matrixes[i]
        
    alpha_graph = nx.from_scipy_sparse_matrix(adj_for_graph, parallel_edges=False, edge_attribute='weight')
    alpha_graph_ig = ig.Graph.from_networkx(alpha_graph)
    
    part = leidenalg.find_partition(alpha_graph_ig, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(gt_partition, part)
    nmi_arr.append(nmi)

  0%|          | 0/8001 [00:00<?, ?it/s]

In [39]:
max_alphas = np.unique(full_alphas[np.nonzero(np.array(nmi_arr) == np.max(nmi_arr))], axis=0)[:100, :]
fusion_k = max_alphas.mean(0)

In [40]:
max_alphas

array([[0.1, 0.1, 0.1, 0.2, 0.2, 0.1, 0.2]])

In [41]:
for net_name in graphs_multiplex_nx:
    net = graphs_multiplex_nx[net_name]
    
    part_gt , _ =  leidenalg.find_partition_multiplex(
    graphs_multiplex_ig[net_name],  # list из слоев графа в формате igraph
    leidenalg.ModularityVertexPartition,  # вставляем leidenalg.ModularityVertexPartition
    n_iterations=2, 
    max_comm_size=0, 
    seed=None)
    
    
    ig_graph = fusion_graph(net, fusion_k)
    part = leidenalg.find_partition(ig_graph, leidenalg.ModularityVertexPartition);
    part = from_part_to_list(part)
    
    nmi = normalized_mutual_info_score(part_gt, part)
    print(nmi)

0.9658820045140976
0.3141744387619212
0.2622125390314791
0.6348276593018142
